In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
import implicit

In [2]:
def calc_mrr(predict, answer):
    for i in range(len(predict)):
        if predict[i] == answer:
            return 1. / (i + 1)
    return 0

max_prediction_len = 100

def calc_score(target_path, predict_path):
    with open(target_path) as f:
        y_true = [int(x.strip()) for x in f.readlines()]

    with open(predict_path) as f:
        y_pred = [[int(x) for x in line.strip().split(' ')] for line in f.readlines()]

    mrr_score = 0
    for (pred, answer) in zip(y_pred, y_true):
        if len(pred) > max_prediction_len:
            raise ValueError('$maximum prediction length is {}, got {}$'.format(max_prediction_len, len(y_pred[i])))
        mrr_score += calc_mrr(pred, answer)

    print(f"MRR@100 = {(mrr_score / len(y_true)):.4f}")

In [3]:
%%time

list_coord = []
list_coord_val = []
target_val = []
np.random.seed(42)  
val_id = np.random.choice(range(10000), size=2500, replace=False)
val_id.sort()

with open('/Users/david/Documents/GitHub/YandexCup_RecSys/train') as f:
    lines = f.readlines()
    idx = 0
    idx_val = 0
    for line in lines:
        tracks = line.strip().split(' ')
        if len(tracks) == 0:
            print(0)
        if len(tracks) == 1:
            print(1)
        for i, track in enumerate(tracks):
            if idx in val_id and i+1 == len(tracks):
                target_val.append(track)
                idx_val += 1
                continue
            if idx in val_id:
                list_coord_val.append((idx_val, track))
            list_coord.append((idx, track))
        idx += 1
        if idx >= 10000:
            break

CPU times: user 4.28 s, sys: 419 ms, total: 4.69 s
Wall time: 5.22 s


In [4]:
%%time

arr_rows = np.array([i[0] for i in list_coord])
arr_cols = np.array([i[1] for i in list_coord]).astype(int)
ones = np.ones(arr_rows.shape[0])
user_item = csr_matrix((ones, (arr_rows, arr_cols)), dtype=int)
#item_user = csr_matrix((ones, (arr_cols, arr_rows)), dtype=int)

arr_rows_val = np.array([i[0] for i in list_coord_val])
arr_cols_val = np.array([i[1] for i in list_coord_val]).astype(int)
ones_val = np.ones(arr_rows_val.shape[0])
user_item_val = csr_matrix((ones_val, (arr_rows_val, arr_cols_val)), dtype=int)

CPU times: user 374 ms, sys: 47.2 ms, total: 421 ms
Wall time: 463 ms


In [5]:
import time

def optimize(factors=30,
             learning_rate=1.00,
             regularization=0.6,
             iterations=30,
             neg_prop=30):
    
    model = implicit.lmf.LogisticMatrixFactorization(factors=factors,
                                                     learning_rate=learning_rate,
                                                     regularization=regularization,
                                                     iterations=iterations,
                                                     neg_prop=neg_prop,
                                                     random_state=42)
    
    model.fit(user_item)
    
    start_time = time.time()
    
    userid = val_id
    recommendations = model.recommend(userid, user_item_val, N=100, filter_already_liked_items=True)
    result = [' '.join(map(str, i)) + '\n' for i in recommendations[0]]
    
    print("--- %s seconds ---" % (time.time() - start_time))
    
    with open('/Users/david/Documents/GitHub/YandexCup_RecSys/impicit_lfm_pred', 'w') as f:
        f.writelines(result)
    
    with open('/Users/david/Documents/GitHub/YandexCup_RecSys/impicit_lfm_target', 'w') as f:
        for i in target_val:
            f.write(f'{i}\n')
            
    calc_score('/Users/david/Documents/GitHub/YandexCup_RecSys/impicit_lfm_target',
               "/Users/david/Documents/GitHub/YandexCup_RecSys/impicit_lfm_pred")

In [11]:
for factors in [5, 15, 30, 50]:
    for iterations in [5, 15, 30, 50]:
        for learning_rate in [0.001, 0.01, 0.1, 1, 10, 100]:
            for regularization in [0.001, 0.01, 0.1, 1, 10, 100]:
                for neg_prop in [5, 15, 30, 50]:
                    print(f'factors: {factors}, iterations: {iterations}, learning_rate: {learning_rate}, regularization: {regularization}, neg_prop: {neg_prop}')
                    optimize(factors=factors,
                             learning_rate=learning_rate,
                             regularization=regularization,
                             iterations=iterations,
                             neg_prop=neg_prop)

factors: 5, iterations: 5, learning_rate: 0.001, regularization: 0.001, neg_prop: 5


  0%|          | 0/5 [00:00<?, ?it/s]

--- 2.396615982055664 seconds ---
MRR@100 = 0.0000
factors: 5, iterations: 5, learning_rate: 0.001, regularization: 0.001, neg_prop: 15


  0%|          | 0/5 [00:00<?, ?it/s]

--- 2.2655558586120605 seconds ---
MRR@100 = 0.0000
factors: 5, iterations: 5, learning_rate: 0.001, regularization: 0.001, neg_prop: 30


...

--- 2.340257167816162 seconds ---
MRR@100 = 0.0002
factors: 5, iterations: 30, learning_rate: 0.1, regularization: 1, neg_prop: 50


  0%|          | 0/30 [00:00<?, ?it/s]

--- 2.405223846435547 seconds ---
MRR@100 = 0.0002
factors: 5, iterations: 30, learning_rate: 0.1, regularization: 10, neg_prop: 5


  0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [12]:
for factors in [10, 20, 30, 50]:
    for iterations in [10, 20, 30, 50]:
        for learning_rate in [1, 10, 100]:
            for regularization in [0.006, 0.06, 0.6, 1, 10, 100]:
                for neg_prop in [5, 15, 30, 50]:
                    print(f'factors: {factors}, iterations: {iterations}, learning_rate: {learning_rate}, regularization: {regularization}, neg_prop: {neg_prop}')
                    optimize(factors=factors,
                             learning_rate=learning_rate,
                             regularization=regularization,
                             iterations=iterations,
                             neg_prop=neg_prop)

factors: 10, iterations: 10, learning_rate: 1, regularization: 0.001, neg_prop: 5


  0%|          | 0/10 [00:00<?, ?it/s]

--- 2.286418914794922 seconds ---
MRR@100 = 0.0006
factors: 10, iterations: 10, learning_rate: 1, regularization: 0.001, neg_prop: 15


  0%|          | 0/10 [00:00<?, ?it/s]

--- 2.3808960914611816 seconds ---
MRR@100 = 0.0018
factors: 10, iterations: 10, learning_rate: 1, regularization: 0.001, neg_prop: 30


...

--- 3.1115400791168213 seconds ---
MRR@100 = 0.0021
factors: 20, iterations: 50, learning_rate: 1, regularization: 100, neg_prop: 15


  0%|          | 0/50 [00:00<?, ?it/s]

--- 3.4244139194488525 seconds ---
MRR@100 = 0.0022
factors: 20, iterations: 50, learning_rate: 1, regularization: 100, neg_prop: 30


  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
for factors in [25, 50, 75]:
    for iterations in [25, 50, 75]:
        for neg_prop in [5, 15, 30, 50]:
            for regularization in [0.001, 0.06, 0.6, 1]:
                print(f'factors: {factors}, iterations: {iterations}, neg_prop: {neg_prop}, regularization: {regularization}')
                optimize(factors=factors,
                         learning_rate=1.00,
                         regularization=regularization,
                         iterations=iterations,
                         neg_prop=neg_prop)

factors: 25, iterations: 25, neg_prop: 5, regularization: 0.001


  0%|          | 0/25 [00:00<?, ?it/s]

--- 3.394413948059082 seconds ---
MRR@100 = 0.0006
factors: 25, iterations: 25, neg_prop: 5, regularization: 0.01


  0%|          | 0/25 [00:00<?, ?it/s]

--- 3.111610174179077 seconds ---
MRR@100 = 0.0010
factors: 25, iterations: 25, neg_prop: 5, regularization: 0.1


...

--- 3.9992709159851074 seconds ---
MRR@100 = 0.0063
factors: 50, iterations: 25, neg_prop: 50, regularization: 0.001


  0%|          | 0/25 [00:00<?, ?it/s]

--- 3.9550869464874268 seconds ---
MRR@100 = 0.0058
factors: 50, iterations: 25, neg_prop: 50, regularization: 0.01


  0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [15]:
for factors in [30, 60, 90]:
    for iterations in [30, 60, 90]:
        for neg_prop in [5, 15, 30, 50]:
            for regularization in [0.6, 1]:
                print(f'factors: {factors}, iterations: {iterations}, neg_prop: {neg_prop}, regularization: {regularization}')
                optimize(factors=factors,
                         learning_rate=1.00,
                         regularization=regularization,
                         iterations=iterations,
                         neg_prop=neg_prop)

factors: 30, iterations: 30, neg_prop: 5, regularization: 0.6


  0%|          | 0/30 [00:00<?, ?it/s]

--- 4.047001123428345 seconds ---
MRR@100 = 0.0071
factors: 30, iterations: 30, neg_prop: 5, regularization: 1


  0%|          | 0/30 [00:00<?, ?it/s]

--- 3.2544469833374023 seconds ---
MRR@100 = 0.0080
factors: 30, iterations: 30, neg_prop: 15, regularization: 0.6


...

--- 3.508333683013916 seconds ---
MRR@100 = 0.0043
factors: 60, iterations: 30, neg_prop: 5, regularization: 1


  0%|          | 0/30 [00:00<?, ?it/s]

--- 4.2966389656066895 seconds ---
MRR@100 = 0.0056
factors: 60, iterations: 30, neg_prop: 15, regularization: 0.6


  0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [16]:
for factors in [50, 75, 100]:
    for iterations in [50, 75, 100]:
        for neg_prop in [5, 15, 30, 50]:
                print(f'factors: {factors}, iterations: {iterations}, neg_prop: {neg_prop}, regularization: {regularization}')
                optimize(factors=factors,
                         learning_rate=1.00,
                         regularization=1.00,
                         iterations=iterations,
                         neg_prop=neg_prop)

factors: 50, iterations: 50, neg_prop: 5, regularization: 0.6


  0%|          | 0/50 [00:00<?, ?it/s]

--- 2.4925220012664795 seconds ---
MRR@100 = 0.0075
factors: 50, iterations: 50, neg_prop: 15, regularization: 0.6


  0%|          | 0/50 [00:00<?, ?it/s]

--- 2.4947900772094727 seconds ---
MRR@100 = 0.0067
factors: 50, iterations: 50, neg_prop: 30, regularization: 0.6


...

--- 3.126235246658325 seconds ---
MRR@100 = 0.0078
factors: 100, iterations: 100, neg_prop: 30, regularization: 0.6


  0%|          | 0/100 [00:00<?, ?it/s]

--- 3.823390007019043 seconds ---
MRR@100 = 0.0054
factors: 100, iterations: 100, neg_prop: 50, regularization: 0.6


  0%|          | 0/100 [00:00<?, ?it/s]

--- 3.717478036880493 seconds ---
MRR@100 = 0.0081


In [17]:
optimize(factors=100,
         learning_rate=1.00,
         regularization=1.00,
         iterations=100,
         neg_prop=10)

  0%|          | 0/100 [00:00<?, ?it/s]

--- 4.323123931884766 seconds ---
MRR@100 = 0.0074


In [18]:
optimize(factors=100,
         learning_rate=1.00,
         regularization=1.00,
         iterations=100,
         neg_prop=100)

  0%|          | 0/100 [00:00<?, ?it/s]

--- 3.7371020317077637 seconds ---
MRR@100 = 0.0068


In [19]:
optimize()

  0%|          | 0/30 [00:00<?, ?it/s]

--- 3.1400530338287354 seconds ---
MRR@100 = 0.0057


In [20]:
for neg_prop in [2, 3, 6, 7]:
        print(f'neg_prop: {neg_prop}')
        optimize(factors=100,
                 learning_rate=1.00,
                 regularization=1.00,
                 iterations=100,
                 neg_prop=neg_prop)

neg_prop: 2


  0%|          | 0/100 [00:00<?, ?it/s]

--- 4.853476047515869 seconds ---
MRR@100 = 0.0147
neg_prop: 3


  0%|          | 0/100 [00:00<?, ?it/s]

--- 5.43638014793396 seconds ---
MRR@100 = 0.0149
neg_prop: 6


  0%|          | 0/100 [00:00<?, ?it/s]

--- 4.45660400390625 seconds ---
MRR@100 = 0.0113
neg_prop: 7


  0%|          | 0/100 [00:00<?, ?it/s]

--- 3.837890863418579 seconds ---
MRR@100 = 0.0105


In [21]:
for neg_prop in [0, 1]:
        print(f'neg_prop: {neg_prop}')
        optimize(factors=100,
                 learning_rate=1.00,
                 regularization=1.00,
                 iterations=100,
                 neg_prop=neg_prop)

neg_prop: 0


  0%|          | 0/100 [00:00<?, ?it/s]

--- 3.9025018215179443 seconds ---
MRR@100 = 0.0024
neg_prop: 1


  0%|          | 0/100 [00:00<?, ?it/s]

--- 3.8038747310638428 seconds ---
MRR@100 = 0.0164


In [22]:
optimize(factors=100,
         learning_rate=1.00,
         regularization=1.00,
         iterations=100,
         neg_prop=4)

  0%|          | 0/100 [00:00<?, ?it/s]

--- 3.9672839641571045 seconds ---
MRR@100 = 0.0122


In [23]:
for factors in [100, 200, 300]:
    for iterations in [100, 200, 300]:
                print(f'factors: {factors}, iterations: {iterations}')
                optimize(factors=factors,
                         learning_rate=1.00,
                         regularization=1.00,
                         iterations=iterations,
                         neg_prop=1)

factors: 100, iterations: 100


  0%|          | 0/100 [00:00<?, ?it/s]

--- 3.99219012260437 seconds ---
MRR@100 = 0.0164
factors: 100, iterations: 200


  0%|          | 0/200 [00:00<?, ?it/s]

--- 4.0477213859558105 seconds ---
MRR@100 = 0.0180
factors: 100, iterations: 300


...

--- 5.27360200881958 seconds ---
MRR@100 = 0.0168
factors: 300, iterations: 200


  0%|          | 0/200 [00:00<?, ?it/s]

--- 6.008830785751343 seconds ---
MRR@100 = 0.0186
factors: 300, iterations: 300


  0%|          | 0/300 [00:00<?, ?it/s]

--- 5.011133193969727 seconds ---
MRR@100 = 0.0182
